In [1]:
import os
import keras
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D
from keras.models import Sequential
from scipy.io import wavfile as wav
import librosa
import librosa.display
import numpy as np
import pandas as pd
import random
import struct
import matplotlib.pyplot as plt
import IPython.display as ipd
import progressbar
import time

In [2]:
data = pd.read_csv('metadata/Screamsound.csv')
valid_data = data[['slice_file_name', 'fold' ,'classID', 'class']][ data['end']-data['start'] > 0 ]
valid_data['path'] = 'fold' + valid_data['fold'].astype('str') + '/' + valid_data['slice_file_name'].astype('str')


In [14]:
from tqdm import tnrange, tqdm_notebook

D=[]
        
for row in tqdm_notebook(valid_data.itertuples()): 
    print(row.path)
    print(row.classID)
    y1, sr1 = librosa.load("audio/" + row.path, duration=2.97)  
    ps = librosa.feature.melspectrogram(y=y1, sr=sr1)
    print(ps.shape)
    if ps.shape != (128, 128): 
            continue
    D.append( (ps, row.classID) )

C:\Users\USER\AppData\Local\Temp\ipykernel_1956\2348564150.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for row in tqdm_notebook(valid_data.itertuples()):


0it [00:00, ?it/s]

fold2/100652-3-0-0.wav
1
(128, 128)
fold2/100652-3-0-1.wav
1
(128, 128)
fold2/100652-3-0-2.wav
1
(128, 128)
fold2/100652-3-0-3.wav
1
(128, 128)
fold2/101415-3-0-2.wav
1
(128, 128)
fold2/101415-3-0-3.wav
1
(128, 128)
fold2/101415-3-0-8.wav
1
(128, 128)
fold1/yisell_sound_2007_11_9_1_31_156634.wav
0
(128, 80)
fold1/yisell_sound_2007_12_18_15_25_315033.wav
0
(128, 13)
fold1/yisell_sound_2007_12_18_15_25_593398.wav
0
(128, 34)
fold1/yisell_sound_2007_12_18_15_26_248752.wav
0
(128, 69)
fold1/yisell_sound_2007_12_18_15_26_444257.wav
0
(128, 84)
fold1/yisell_sound_2007_12_18_15_27_063699.wav
0
(128, 62)
fold1/yisell_sound_2007_12_18_15_27_324809.wav
0
(128, 94)
fold1/yisell_sound_2007_12_18_15_27_543156.wav
0
(128, 128)
fold1/yisell_sound_2007_12_18_15_29_067921.wav
0
(128, 128)
fold1/yisell_sound_2007_12_18_15_29_33722.wav
0
(128, 128)
fold1/yisell_sound_2007_12_18_15_30_231364.wav
0
(128, 128)
fold1/yisell_sound_2014080511023849341_66366.wav
0
(128, 88)
fold1/yisell_sound_201408051232386395

In [4]:
import hickle as hkl 
data = D

# Dump data to file
hkl.dump(data, 'new_data_file.hkl')

In [5]:
# Load data from file
D = hkl.load('new_data_file.hkl')

In [6]:
from keras.utils import np_utils
dataset = D
random.shuffle(dataset)

train = dataset[:13]
test = dataset[13:]
print('dataset:', len(dataset))
print('train:', len(train))
print('test:', len(test))

X_train, y_train = zip(*train)
X_test, y_test = zip(*test)

X_train = np.array([x.reshape( (128, 128, 1) ) for x in X_train])
X_test = np.array([x.reshape( (128, 128, 1) ) for x in X_test])


y_train = np.array(keras.utils.np_utils.to_categorical(y_train, 10))
y_test = np.array(keras.utils.np_utils.to_categorical(y_test, 10))

dataset: 19
train: 13
test: 6


In [7]:
model = Sequential()
input_shape=(128, 128, 1)

model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(10))
model.add(Activation('softmax'))
 

In [8]:
model.compile(
    optimizer="Adam",
    loss="categorical_crossentropy",
    metrics=['accuracy'])

model.fit(
    x=X_train, 
    y=y_train,
    epochs=24,
    batch_size=128,
    validation_data= (X_test, y_test))

score = model.evaluate(
    x=X_test,
    y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/24
1/1 [==============================] - 1s 1s/step - loss: 39.6599 - accuracy: 0.0000e+00 - val_loss: 4.2205 - val_accuracy: 0.3333
Epoch 2/24
1/1 [==============================] - 0s 199ms/step - loss: 3.7582 - accuracy: 0.6923 - val_loss: 1.7002 - val_accuracy: 0.6667
Epoch 3/24
1/1 [==============================] - 0s 205ms/step - loss: 0.3189 - accuracy: 0.9231 - val_loss: 1.8145 - val_accuracy: 0.6667
Epoch 4/24
1/1 [==============================] - 0s 159ms/step - loss: 0.8300 - accuracy: 0.8462 - val_loss: 2.7192 - val_accuracy: 0.6667
Epoch 5/24
1/1 [==============================] - 0s 198ms/step - loss: 0.0974 - accuracy: 0.9231 - val_loss: 3.2675 - val_accuracy: 0.6667
Epoch 6/24
1/1 [==============================] - 0s 161ms/step - loss: 3.4740 - accuracy: 0.8462 - val_loss: 4.2272 - val_accuracy: 0.6667
Epoch 7/24
1/1 [==============================] - 0s 138ms/step - loss: 0.3431 - accuracy: 0.9231 - val_loss: 5.1500 - val_accuracy: 0.6667
Epoch 8/24
1/1 [==

In [9]:
model.save('ScreamDemo1.h5') 
print('Model exported and finished')

Model exported and finished
